### Pinetree Errors

In [15]:
import pandas as pd
import numpy as np
import pinetree as pt
from sklearn.metrics import mean_squared_error 
base_dir = "/Users/tanviingle/Documents/Wilke/phix174/"

def run_pt(gen, pA, pB, pD):
    print(gen, pA)
    print("Defining PhiX-174 genome")

    # Create host cell & genome
    CELL_VOLUME = 1.1e-15 # from T7
    model = pt.Model(cell_volume=CELL_VOLUME)
    phage = pt.Genome(name="phix_174", length=5386)

    # Read genomic coordinates from csv into dataframe
    genomic_coords = pd.read_csv(base_dir + "output/" + "genomic_coords.csv")
    print(genomic_coords.at[0, "type"])


    # Add genomic ns (loop through ^ df); hardcode necessary strengths according to preomtimized_model
    ## for length of genomic_coords, add elements
    #for n in genomic_coords:{
    n = 0
    while(n < len(genomic_coords)):

        if genomic_coords.at[n, "type"] == "gene": 
            phage.add_gene(name= genomic_coords.at[n, "type"] + "_" + genomic_coords.at[n, "name"], 
                           start= genomic_coords.at[n, "new_start"], 
                           stop= genomic_coords.at[n, "new_end"],
                           rbs_start=genomic_coords.at[n, "new_start"], 
                           rbs_stop=genomic_coords.at[n, "new_start"] + 15, rbs_strength=1e7) 

        elif genomic_coords.at[n, "type"] == "promoter" and genomic_coords.at[n, "name"] == "A":
            phage.add_promoter(name= genomic_coords.at[n, "type"] + "_" + genomic_coords.at[n, "name"], 
                               start= genomic_coords.at[n, "new_start"], 
                               stop= genomic_coords.at[n, "new_end"]+1,
                               interactions={"ecolipol": pA})

        elif genomic_coords.at[n, "type"] == "promoter" and genomic_coords.at[n, "name"] == "B1":
            phage.add_promoter(name= genomic_coords.at[n, "type"] + "_" + genomic_coords.at[n, "name"], 
                               start= genomic_coords.at[n, "new_start"], 
                               stop= genomic_coords.at[n, "new_end"],
                               interactions={"ecolipol": pB})

        elif genomic_coords.at[n, "type"] == "promoter" and genomic_coords.at[n, "name"] == "D":
            phage.add_promoter(name= genomic_coords.at[n, "type"] + "_" + genomic_coords.at[n, "name"], 
                               start= genomic_coords.at[n, "new_start"], 
                               stop= genomic_coords.at[n, "new_end"],
                               interactions={"ecolipol": pD})

        else:
            print("ignoring pB2")

        n = n+1

    print("all genes and promoters added")

    # Add terminators manually 
    phage.add_terminator(name="terminator_J", start=2434, stop=2435, # Right before gene F start=2404, stop=3687,
                       efficiency={"ecolipol": 0.7}) # 0.7; 2402-2403
    phage.add_terminator(name="terminator_F", start=3754, stop=3755, # Right before gene G start=3798, stop=4325
                     efficiency={"ecolipol": 0.8}) # 0.8; 3796-3797
    phage.add_terminator(name="terminator_G", start=4392, stop=4393, # Right before gene H start=4334, stop=5320
                     efficiency={"ecolipol": 0.6}) # 0.6; 4332-4333
    phage.add_terminator(name="terminator_H", start=47, stop=48, # Right after promoter A
                     efficiency={"ecolipol": 0.3}) # 0.3; 5321-5322

    print("all terminators added")

    # Register genome after promoters/terminators are added
    model.register_genome(phage)
    print("genome is registered")

    # Define interactions
    print("Defining Polymerases & Interactions")
    # Add polymerases & species
    model.add_polymerase(name="ecolipol", speed=35, footprint=35, copy_number=0)
    model.add_species("bound_ecolipol", 1800)  # initialization
    model.add_species("ecoli_genome", 0)
    model.add_species("ecoli_transcript", 0)
    model.add_reaction(1e6, ["ecolipol", "ecoli_genome"], ["bound_ecolipol"]) # 1e7
    model.add_reaction(0.04, ["bound_ecolipol"], ["ecolipol", "ecoli_genome", "ecoli_transcript"])
    model.add_ribosome(10, 30, 100)
    model.add_species("bound_ribosome", 100)
    model.seed(34)

    # Run simulation
    print("running simulation")
    model.simulate(time_limit=2000, time_step=5, output= base_dir + "output/overlap_poly_error.tsv") # TODO change limit
    print("Simulation successful!")

#### ERROR 1: Overlapping polymerases
Pinetree output reads: Polymerase __ribosome (start: 2317, stop: 2326, index: 2) is overlapping polymerase __ribosome (start: 2323, stop: 2332, index: 3) by more than one position on polymer __rna

In [17]:
gen = 0
pA = 2e5
#pB = 5e7
#pD = 2e6
pB = 5e5
pD = 5e6

run_pt(gen, pA, pB, pD)

0 200000.0
Defining PhiX-174 genome
gene
ignoring pB2
all genes and promoters added
all terminators added
genome is registered
Defining Polymerases & Interactions
running simulation
Simulation successful!


In [ ]:
gen = 0
pA = 2e5
pB = 6.25*pA
pD = 5.25*pA

run_pt(gen, pA, pB, pD)